In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import poisson, norm
from scipy.special import expit
from functools import lru_cache

<center><h1>Identifying pumps</h1></center>

## Approach

In order to determine which residences had pumps, I needed a way to distinguish pump power consumption patterns from those of other household appliances.  I hypothesized that a typical pump
<ol>
<li>consumes more power than the typical appliance</li>
<li>mostly turns on or off suddenly, so could cause power "spikes" that not all other appliances would produce</li>
</ol>

These hypotheses led me to the following procedure:
<ol>
<li>estimate the distribution of a typical residence's change in power usage in an arbitrary time interval, both with and without a pump</li>
<li>calculate, for each sample residence, the change in power usage between each adjacent pair of timestamps</li>
<li>compare each sample residence's changes in power usage with the distributions I estimated, and use the comparison to estimate the probability that the residence has a pump</li>
<li>suggest the three residences with highest probabilities of containing pumps as the ones with pumps</li>
</ol>

As I implement this approach below, I'll make many assumptions, most of which are probably quite inaccurate.  But I think the approach establishes a good general framework, one that could support a very reasonable model for discriminating between households with and without pumps.

## Notation

Let
* $X^{(t)}$ be the continuous random variable measuring a household's change in power consumption in a random interval of length $t$ seconds
* $\mbox{pump}$ be the binary random variable corresponding to the presence of a pump

I use $p$ to denote the probability density function (pdf).  So $p(X^{(t)} \mid \mbox{pump})$ expresses the pdf of $X^{(t)}$ in a house with a pump, and $p(X^{(t)} \mid \neg \mbox{pump})$ expresses the pdf of $X^{(t)}$ in a house with no pump.

I will represent probability distributions in code as pandas `Series` objects.  Even when they don't have compact support, they'll be nearly zero at sufficiently large values, so the representation in large arrays is safe.  I'll index these `Series` objects by the array `L` defined in the next code block.

In [2]:
INFINITY = 10000
L = np.arange(-INFINITY, INFINITY)

For example, the following `Series` would represent a standard normal distribution:

In [0]:
pd.Series(norm.pdf(L, loc=0, scale=1), index=L)

## Parse the data

Here is the code I use to read the data and calculate the change in power consumption between each consecutive pair of measurements.

In [3]:
file_numbers = (12, 13, 14, 20, 22, 40)

data = {i: pd.read_csv('UsageData_{0}.csv'.format(i)) for i in file_numbers}

def calculate_diffs(df):
    ary = df[['timestamp', 'amount']].values # cast to array to ignore index when subtracting
    d = pd.DataFrame(ary[1:,:] - ary[:-1,:], columns=['timedelta', 'wattdelta'])
    d['wattdelta'] = (d['wattdelta'] * 1000).astype(int)
    return d

diffs = {i: calculate_diffs(df) for i, df in data.items()}

`diffs` will be used again in subsequent sections.

## Distribution of power changes without a pump

This section explains my approach to estimate the distribution of changes in power in a house with no pump.

##### Assumptions

Make the following assumptions:
<ol>
<li> The distribution of changes in power associated with a single (non-pump) device being switched on or off is normal, with standard deviation of 100 watts:
$$
\mbox{change in power when one device is switched on or off} \sim N(0, \sigma ^2) \\
\sigma = 100\mbox{ watts}
$$
</li>
<li>Devices are switched on or off independently of each other, and on average, two devices are switched on or off per hour.</li>
<li>In the absence of a pump, all changes in power consumption can be attributed to a non-pump device switching on and off.</li>
</ol>

##### Derivation of formula

Using assumption 1 and the usual formula for the sum of normally distributed random variables, we see that the distribution of changes in power associated with $n$ household devices being switched on or off is
$$
\mbox{change in power when }n\mbox{ devices are switched on or off} \sim \begin{cases}N(0, n\sigma ^2) & \mbox{if }n > 0 \\
\delta(0) & \mbox{if }n = 0
\end{cases}\\
\sigma = 100\mbox{ watts}
$$
where $\delta(0)$ is the Dirac delta distribution supported at $0$.  Here is code that generates this distribution.

In [4]:
def pdf_given_n_devices_get_switched(n):
    if n == 0:
        return pd.Series(L == 0, index=L).astype(float)
    else:
        loc = 0
        std = 100
        return pd.Series(norm.pdf(L, loc, np.sqrt(n) * std), index=L)

It follows from assumption 2 that the number of "device switch" events, consisting of any non-pump device being turned on or off, has a Poisson distribution:
$$
P(n\mbox{ devices switched on or off in }t\mbox{ hours}) = \frac{\lambda ^n e^{-\lambda}}{n!} \\
\lambda = (2\mbox{ events/hour}) \cdot t
$$
Here is the translation to code:

In [5]:
def probability_n_devices_get_switched(n, interval_seconds):
    hourly_frequency = 2
    return poisson.pmf(n, hourly_frequency * interval_seconds / 3600)

It follows that distribution of changes in power attributable to non-pump devices is

\begin{align*}
p(X^{(t)} \mid \neg \mbox{pump}) &= e^{-\lambda} \delta(0) + \sum _{n > 0}\frac{\lambda ^n e^{-\lambda}}{n!}N(0, n\sigma ^2)
\end{align*}

The `pdf_given_no_pump` function returns this distribution.

In [6]:
@lru_cache(maxsize=5) # memoize the result
def pdf_given_no_pump(interval_seconds):
    max_number_of_devices_reasonably_switched = int(interval_seconds / 60)
    return sum([probability_n_devices_get_switched(n, interval_seconds) *
                pdf_given_n_devices_get_switched(n)
                for n in range(max_number_of_devices_reasonably_switched)])

## Distribution of power changes with a pump

This section explains my approach to estimating the distribution of changes in power in a house with a pump.

##### Assumptions

Assume that in a house with a pump,
1. the pump is, on average, switched on or off 10 times per day, and it is equally likely to be switched on or off at any time
2. the pump's power consumption is uniformly distributed between 100 and 750 watts

##### Derivation of formula

Here is a function that generates the distribution
$$
p(X \mid \mbox{one pump turns on/off once})
$$
of changes in power consumption associated with a pump turning on or off once.  It is a direct translation of the second assumption above.

In [7]:
def pdf_given_one_pump_switch():
    min_pump_wattage = 100
    max_pump_wattage = 750
    normalization = 0.5 / (max_pump_wattage - min_pump_wattage)
    support = ((np.abs(L) <= max_pump_wattage) & (np.abs(L) >= min_pump_wattage))

    return pd.Series(support * normalization, index=L)

In a given time interval, we observe a change in power consumption associated with the pump if and only if the pump is switched on or off an odd number of times.  It follow from assumption 1 that the probability of an odd number of switches occurring is
$$
P(\mbox{odd number of switches in }t\mbox{ hours}) = \frac{1 - e^{-2 t\rho}}{2} \\
\rho = \frac{10}{24}\mbox{ switches/hour}
$$
(See <a href="http://math.stackexchange.com/questions/472431/probability-that-an-integer-number-having-poisson-distribution-is-even">here</a> for a derivation of this formula.)  Here is a corresponding Python function:

In [8]:
def probability_of_odd_num_pump_switches(interval_seconds):
    pump_daily_frequency = 10
    pump_interval_frequency = interval_seconds * pump_daily_frequency / 24 / 3600

    return (1 - np.exp(-2*pump_interval_frequency)) / 2

Therefore the distribution of changes in power consumption attributable to a pump is

\begin{align*}
p(X^{(t)}\mid \mbox{one pump, no other devices}) =& P(\mbox{odd number of switches in }t\mbox{ hours})\cdot p(X \mid \mbox{one pump turns on or off exactly once}) +\\
& P(\mbox{even number of switches in }t\mbox{ hours})\cdot \delta(0)
\end{align*}

In code:

In [9]:
def pdf_from_pump_alone(interval_seconds):
    probability_of_change = probability_of_odd_num_pump_switches(interval_seconds)

    return (probability_of_change * pdf_given_one_pump_switch() +
            (1 - probability_of_change) * pd.Series(L == 0, index=L))

The distribution of changes in power in a house with a pump is then
$$
p(X^{(t)} \mid \mbox{pump}) = p(X^{(t)} \mid \neg\mbox{pump}) + p(X^{(t)}\mid \mbox{one pump, no other devices})
$$

In [10]:
@lru_cache(maxsize=5)
def pdf_given_pump(interval_seconds):
    no_pump_pdf = pdf_given_no_pump(interval_seconds)
    pump_pdf = pdf_from_pump_alone(interval_seconds)

    # Recall that the convolution of distributions of random variables is the distribution of their
    # sum.
    return pd.Series(np.convolve(no_pump_pdf, pump_pdf)[INFINITY:3*INFINITY], index=L)

## Calculating the probability of having a pump

In the previous sections we calculated the distributions $p(X^{(t)} \mid \mbox{pump})$ and $p(X^{(t)} \mid \neg \mbox{pump})$.  We will now see how to use these values to calculate the probability $P(\mbox{pump} \mid \{x_i\})$, the probability that a sequence of observations $x_i$ of changes in power consumption is obtained from a house with a pump.

##### Assumptions

Assume that changes in power consumption are independent.  This is, of course, not exactly true (e.g. because a device cannot be turned on twice in a row), but it is required for the math to be tractable.

##### Derivation of formula

\begin{align*}
P(\mbox{pump} \mid \{x_i\}) &= \frac{P(\mbox{pump}) p(\{x_i\} \mid \mbox{pump})}{p(\{x_i\})} & \mbox{by Bayes' theorem} \\
&= \frac{P(\mbox{pump}) \prod \limits_i p(x_i \mid \mbox{pump})}{p(\{x_i\})} & \mbox{by independence of observations} \\
\end{align*}

and similarly,
$$
P(\neg \mbox{pump} \mid \{x_i\}) = \frac{P(\neg \mbox{pump}) \prod \limits_i p(x_i \mid \neg \mbox{pump})}{p(\{x_i\})}
$$
Since we know that three out of the six time series come from houses with pumps,
$$
P(\mbox{pump}) = P(\neg \mbox{pump}) = 0.5
$$
Combining everything,
$$
\frac{P(\mbox{pump} \mid \{x_i\})}{P(\neg \mbox{pump} \mid \{x_i\})} = \frac{\prod \limits_i p(x_i \mid \mbox{pump})}{\prod \limits_i p(x_i \mid \neg \mbox{pump})}
$$
To avoid numerical underflow, we will calculate the logarithm of both sides of the equation:

\begin{align*}
\log \frac{P(\mbox{pump} \mid \{x_i\})}{P(\neg \mbox{pump} \mid \{x_i\})} &= \log \frac{\prod \limits_i p(x_i \mid \mbox{pump})}{\prod \limits_i p(x_i \mid \neg \mbox{pump})} \\
&= \sum \limits_i \log p(x_i \mid \mbox{pump}) - \sum \limits_i \log p(x_i \mid \neg \mbox{pump})
\end{align*}

The expression on the left can be rewritten in terms of the log-odds function $\mbox{logit }p := \log \frac{p}{1 - p}$, so
$$
\mbox{logit } P(\mbox{pump} \mid \{x_i\}) = \sum \limits_i \log p(x_i \mid \mbox{pump}) - \sum \limits_i \log p(x_i \mid \neg \mbox{pump})
$$
Since $\mbox{logit}$ is an increasing function, the residences with the highest probabilities of having pumps are the same as the residences with the highest values of $\mbox{logit } P(\mbox{pump} \mid \{x_i\})$, so it suffices for us to rank residences by $\mbox{logit } P(\mbox{pump} \mid \{x_i\})$.  On the other hand, the terms in the right-hand side of the equation above are all samples from distributions that we estimated explicitly in previous sections.  So the right-hand side can be calculated.  Here are the functions to calculate it.

In [11]:
def log_likelihood_given_pump(d):
    def likelihood_of_single_measurement(measurement):
        return pdf_given_pump(measurement['timedelta'])[measurement['wattdelta']]

    likelihoods = d.apply(likelihood_of_single_measurement, axis=1)

    return np.log(likelihoods).sum()


def log_likelihood_given_no_pump(d):
    def likelihood_of_single_measurement(measurement):
        return pdf_given_no_pump(measurement['timedelta'])[measurement['wattdelta']]

    likelihoods = d.apply(likelihood_of_single_measurement, axis=1)

    return np.log(likelihoods).sum()


def log_odds_of_pump(d):
    return log_likelihood_given_pump(d) - log_likelihood_given_no_pump(d)

## Result

Now we are ready to guess which residences have pumps.

In [12]:
for i, df in diffs.items():
    log_odds = log_odds_of_pump(df)
    print('log-odds of set {0} having a pump is {1}'.format(i, log_odds))

log-odds of set 14 having a pump is 16631.001449021205
log-odds of set 13 having a pump is 11512.570354557189
log-odds of set 12 having a pump is 4514.085204111645
log-odds of set 40 having a pump is 26986.309899553104
log-odds of set 22 having a pump is 4760.373489192862
log-odds of set 20 having a pump is -1577.6427598666924


Therefore, sets 13, 14, and 40 are most likely to have pumps.